In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [12]:

# Define constants
BASE_DIR = './dataset/train'

# * Number of times we are going to run through the entire dataset.
EPOCHS = 10  
# * the image size that we are going to set the images in the dataset to.
IMAGE_SIZE = 224
# * the number of images we are inputting into the neural network at once.
BATCH_SIZE = 64
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

RESULT_LABELS = 'results/labels.txt'


In [3]:
# Set up data generator with data augmentation techniques such as rotation, shifting and flipping
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # rescale pixel values to [0,1]
    validation_split=0.2,  # reserve some data for validation
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
)

In [5]:
# Generate training data
train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'  # specify this is training data
)

Found 2510 images belonging to 36 classes.


In [6]:
# Generate validation data
val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'  # specify this is validation data
)

Found 605 images belonging to 36 classes.


In [7]:
# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # monitor the validation loss
    patience=5,  # number of epochs with no improvement after which training will be stopped
    verbose=1,  # report the early stopping events
    restore_best_weights=True  # restore the best weights from the monitored quantity
)

In [8]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # monitor the validation loss
    factor=0.2,  # factor by which the learning rate will be reduced
    patience=3,  # number of epochs with no improvement after which learning rate will be reduced
    verbose=1,  # report the lr reduction events
    min_lr=0.0001  # lower bound on the learning rate
)

In [13]:
# Print and save the class indices
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open(RESULT_LABELS, 'w') as f:
    f.write(labels)

{'apple': 0, 'banana': 1, 'beetroot': 2, 'bell pepper': 3, 'cabbage': 4, 'capsicum': 5, 'carrot': 6, 'cauliflower': 7, 'chilli pepper': 8, 'corn': 9, 'cucumber': 10, 'eggplant': 11, 'garlic': 12, 'ginger': 13, 'grapes': 14, 'jalepeno': 15, 'kiwi': 16, 'lemon': 17, 'lettuce': 18, 'mango': 19, 'onion': 20, 'orange': 21, 'paprika': 22, 'pear': 23, 'peas': 24, 'pineapple': 25, 'pomegranate': 26, 'potato': 27, 'raddish': 28, 'soy beans': 29, 'spinach': 30, 'sweetcorn': 31, 'sweetpotato': 32, 'tomato': 33, 'turnip': 34, 'watermelon': 35}


In [ ]:
# Define the device strategy
strategy = tf.distribute.OneDeviceStrategy("GPU:0")

# Define the model within the strategy scope
with strategy.scope():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMAGE_SHAPE,
        include_top=False,  # exclude the top layer
    )
    base_model.trainable = False  # freeze the base model

    # Define the custom top layers
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Conv2D(64, 3, activation='relu'),  # increased filter size for more complex patterns
        tf.keras.layers.BatchNormalization(),  # normalize the activations of the previous layer at each batch
        tf.keras.layers.Dropout(0.2),  # randomly set a fraction rate of input units to 0 at each update during training
        tf.keras.layers.Conv2D(64, 3, activation='relu'),  # another convolutional layer
        tf.keras.layers.BatchNormalization(),  # batch normalization layer
        tf.keras.layers.GlobalAveragePooling2D(),  # apply average pooling on the spatial dimensions until each spatial dimension is one
        tf.keras.layers.Dense(128, activation='relu'),  # dense layer with more neurons
        tf.keras.layers.Dense(36, activation='softmax')  # final softmax layer
    ])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),  # use Adam optimizer with initial learning rate 0.001
        loss='categorical_crossentropy',  # use categorical cross entropy as the loss function
        metrics=['accuracy']  # use accuracy as the metric
    )

### Train the model

In [16]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]  # add the callbacks to the training process
)

Epoch 1/10


2023-06-26 14:36:03.361328: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021TensorDataset:187"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}

2023-06-26 14:36:03.388039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is e

40/40 [==============================] - ETA: 0s - loss: 0.1893 - accuracy: 0.9406

2023-06-26 14:36:57.017964: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021TensorDataset:204"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}

2023-06-26 14:36:57.045225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is e

40/40 [==============================] - 69s 2s/step - loss: 0.1893 - accuracy: 0.9406 - val_loss: 0.5960 - val_accuracy: 0.8116 - lr: 2.0000e-04
Epoch 2/10


...

40/40 [==============================] - 66s 2s/step - loss: 0.1428 - accuracy: 0.9558 - val_loss: 0.5672 - val_accuracy: 0.8149 - lr: 1.0000e-04
Epoch 7/10


2023-06-26 14:42:46.399839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 14:42:46.403427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - ETA: 0s - loss: 0.1355 - accuracy: 0.9618

2023-06-26 14:43:38.152308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 14:43:38.155481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 14:43:38.179016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 14:43:38.182034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Restoring model weights from the end of the best epoch: 2.
40/40 [==============================] - 67s 2s/step - loss: 0.1355 - accuracy: 0.9618 - val_loss: 0.5823 - val_accuracy: 0.8215 - lr: 1.0000e-04
Epoch 7: early stopping


### Save the trained model

In [18]:
# Save the model
saved_model_dir = 'results'
tf.saved_model.save(model, saved_model_dir)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
with open('results/model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: results/assets


INFO:tensorflow:Assets written to: results/assets
2023-06-26 14:46:17.957353: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-26 14:46:17.957366: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-26 14:46:17.957508: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: results
2023-06-26 14:46:17.968685: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-06-26 14:46:17.968694: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: results
2023-06-26 14:46:17.999259: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-26 14:46:18.281060: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: results
2023-06-26 14:46:18.378084: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: su